In [57]:
import pandas as pd
import numpy as np
import ast

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

### Binary prediction Metrics

In [3]:
# Performance measures

def prediction_measures(y_test,y_pred):
    performance = classification_report(y_test,y_pred,output_dict=True)
    return performance['depression'],performance['normal'],performance['accuracy']


In [4]:
# All measurements

def all_measures(predsensitive, truesensitive, predother, trueother, name='test',single=False):
    score_dict = {}
    score_dict['predictor'] = name
    # Metrics for sensitive group
    depression_performance, normal_performance ,accuracy = prediction_measures(predsensitive,truesensitive)
    score_dict['depressionSensPrecision'] = depression_performance['precision']
    score_dict['depressionSensRecall'] = depression_performance['recall']
    score_dict['depressionSensF1'] = depression_performance['f1-score']
    score_dict['depressionSensSupport'] = depression_performance['support']
    score_dict['normalSensPrecision'] = normal_performance['precision']
    score_dict['normalSensRecall'] = normal_performance['recall']
    score_dict['normalSensF1'] = normal_performance['f1-score']
    score_dict['normalSensSupport'] = normal_performance['support']
    score_dict['accuracySens'] = accuracy
    if single == False:
        # Metrics for other group
        depression_performance, normal_performance ,accuracy = prediction_measures(predother,trueother)
        score_dict['depressionOtherPrecision'] = depression_performance['precision']
        score_dict['depressionOtherRecall'] = depression_performance['recall']
        score_dict['depressionOtherF1'] = depression_performance['f1-score']
        score_dict['depressionOtherSupport'] = depression_performance['support']
        score_dict['normalOtherPrecision'] = normal_performance['precision']
        score_dict['normalOtherRecall'] = normal_performance['recall']
        score_dict['normalOtherF1'] = normal_performance['f1-score']
        score_dict['normalOtherSupport'] = normal_performance['recall']
        score_dict['accuracyOther'] = accuracy
        # Metrics for total
        depression_performance, normal_performance ,accuracy = prediction_measures(pd.concat([predsensitive,predother]),pd.concat([truesensitive,trueother]))
        score_dict['depressionTotalPrecision'] = depression_performance['precision']
        score_dict['depressionTotalRecall'] = depression_performance['recall']
        score_dict['depressionTotalF1'] = depression_performance['f1-score']
        score_dict['depressionTotalSupport'] = depression_performance['support']
        score_dict['normalTotalPrecision'] = normal_performance['precision']
        score_dict['normalTotalRecall'] = normal_performance['recall']
        score_dict['normalTotalF1'] = normal_performance['f1-score']
        score_dict['normalTotalSupport'] = normal_performance['recall']
        score_dict['accuracyTotal'] = accuracy
        # Fairness metrics between groups
        TNS, FPS, FNS, TPS = confusion_matrix(truesensitive, predsensitive).ravel()
        TNO, FPO, FNO, TPO = confusion_matrix(trueother,predother).ravel()
        score_dict['statisticalParity'] = ((TPS+FPS)/len(truesensitive)) / ((TPO+FPO)/len(trueother))
        score_dict['equalOpportunity'] = (TPS / (TPS+FNS)) / (TPO / (TPO+FNO))
        score_dict['equalisedOdds'] =  ((TPS / (TPS+FNS)) + (TNS / (TNS+FPS))) / ((TPO / (TPO+FNO))+ (TNO) / (TNO+FPO))
        score_dict['equalAccuracy'] = (accuracy_score(truesensitive,predsensitive)/accuracy_score(trueother,predother))
    return score_dict

In [ ]:


options = ['w2vec','ruleBased','DictionaryBased','TFIDF500','TFIDFEX500','TFIDFIN500','blingfire','reweightedblingfire','nltk','reweightednltk','spacysm','reweightedspacysm','spacylg','reweightedspacylg','spacytrf','reweightedspacytrf']


for name in options:
    df = pd.read_json(f'data/predictionData/{name}Pred.json', lines=True, orient='records')

    pred_male = df.loc[(df['gender'] == 'm') & (df['split'] == 'test'), ['prediction']]
    pred_female = df.loc[(df['gender'] == 'f') & (df['split'] == 'test'), ['prediction']]
    y_testm = df.loc[(df['gender'] == 'm') & (df['split'] == 'test'), ['label']]
    y_testf = df.loc[(df['gender'] == 'f') & (df['split'] == 'test'), ['label']]

    score_dict = all_measures(pred_female,y_testf,pred_male,y_testm, name)

    try:
        df = pd.read_csv('data/results/depressiontestresults.csv')
    except:
        df = pd.DataFrame()

    df_new = pd.DataFrame(score_dict,index=[len(df)])

    df = pd.concat([df,df_new],ignore_index=True)

    df.to_csv('data/results/depressiontestresults.csv', index=False)

### Symptom prediction results

In [74]:
ytruem = pd.read_csv('data/annotationData/MaleAnnotationsetFilled.csv', sep=';')
ytruef = pd.read_csv('data/annotationData/FemaleAnnotationsetFilled.csv', sep=';')
ypredm = pd.read_csv('data/annotationData/malePredictionsetFilled.csv')
ypredf = pd.read_csv('data/annotationData/femalePredictionsetFilled.csv')

df = ytruef

for x in range(len(df)):
    for i in range(1,20,2):
        try:
            obj = ast.literal_eval(ypredm[str(i)][x])
            print((list(obj.keys())[0]))
        except: continue


Anxious_Mood
Depressed_Mood
flight_of_ideas
intrusion_symptoms
obsession
Anger_Irritability
Depressed_Mood
Worthlessness_and_guilty
Anxious_Mood
pessimism
Depressed_Mood
do_things_easily_get_painful_consequences
drastical_shift_in_mood_and_energy
Depressed_Mood
Worthlessness_and_guilty
pessimism
Worthlessness_and_guilty
pessimism
Anxious_Mood
sleep_disturbance
Depressed_Mood
Worthlessness_and_guilty
pessimism
Autonomic_symptoms
Cardiovascular_symptoms
Respiratory_symptoms
Suicidal_ideas
do_things_easily_get_painful_consequences
Depressed_Mood
Suicidal_ideas
Worthlessness_and_guilty
do_things_easily_get_painful_consequences
Depressed_Mood
Worthlessness_and_guilty
pessimism
Autonomic_symptoms
Cardiovascular_symptoms
Respiratory_symptoms
Depressed_Mood
Worthlessness_and_guilty
pessimism
Depressed_Mood
diminished_emotional_expression
Anxious_Mood
Depressed_Mood
Hyperactivity_agitation
drastical_shift_in_mood_and_energy
panic_fear
Anger_Irritability
flight_of_ideas
intrusion_symptoms
Anxiou